# Building a Neural Network from Scratch
Our NN will have a simple two-layer architecture. Input layer $a^{[0]}$ will have 784 units corresponding to the 784 pixels in each 28x28 input image. A hidden layer $a^{[1]}$ will have 10 units with ReLU activation, and finally our output layer $a^{[2]}$ will have 10 units corresponding to the ten digit classes with softmax activation.

**Forward propagation**

$$Z^{[1]} = W^{[1]} X + b^{[1]}$$
$$A^{[1]} = g_{\text{ReLU}}(Z^{[1]}))$$
$$Z^{[2]} = W^{[2]} A^{[1]} + b^{[2]}$$
$$A^{[2]} = g_{\text{softmax}}(Z^{[2]})$$

**Backward propagation**

$$dZ^{[2]} = A^{[2]} - Y$$
$$dW^{[2]} = \frac{1}{m} dZ^{[2]} A^{[1]T}$$
$$dB^{[2]} = \frac{1}{m} \Sigma {dZ^{[2]}}$$
$$dZ^{[1]} = W^{[2]T} dZ^{[2]} .* g^{[1]\prime} (z^{[1]})$$
$$dW^{[1]} = \frac{1}{m} dZ^{[1]} A^{[0]T}$$
$$dB^{[1]} = \frac{1}{m} \Sigma {dZ^{[1]}}$$

**Parameter updates**

$$W^{[2]} := W^{[2]} - \alpha dW^{[2]}$$
$$b^{[2]} := b^{[2]} - \alpha db^{[2]}$$
$$W^{[1]} := W^{[1]} - \alpha dW^{[1]}$$
$$b^{[1]} := b^{[1]} - \alpha db^{[1]}$$

**Vars and shapes**

Forward prop

- $A^{[0]} = X$: 784 x m
- $Z^{[1]} \sim A^{[1]}$: 10 x m
- $W^{[1]}$: 10 x 784 (as $W^{[1]} A^{[0]} \sim Z^{[1]}$)
- $B^{[1]}$: 10 x 1
- $Z^{[2]} \sim A^{[2]}$: 10 x m
- $W^{[1]}$: 10 x 10 (as $W^{[2]} A^{[1]} \sim Z^{[2]}$)
- $B^{[2]}$: 10 x 1

Backprop

- $dZ^{[2]}$: 10 x m ($~A^{[2]}$)
- $dW^{[2]}$: 10 x 10
- $dB^{[2]}$: 10 x 1
- $dZ^{[1]}$: 10 x m ($~A^{[1]}$)
- $dW^{[1]}$: 10 x 10
- $dB^{[1]}$: 10 x 1

# 0. Importing packages

In [29]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

path = r'digit-recognizer\train.csv'

data = pd.read_csv(path)


Loaded the data into a pandas dataframe.

In [30]:
data.head()


,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Converting the data from a dataframe to a numpy array.

In [31]:
data = np.array(data)


Making sure that the model isn't overfitted, i.e., the model makes fairly accurate predictions for the training data but isn't generalised for the data it's supposed to have a high accuracy for. Setting aside a portion of the training data to perform cross-validation on to avoid overfitting.

Shuffling the data before we split the data into dev and training data. Note, `np.random.shuffle()` permutes the sequence in place.

In [32]:
np.random.shuffle(data)


In [33]:
data


array([[9, 0, 0, ..., 0, 0, 0],
       [6, 0, 0, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       ...,
       [6, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [3, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [34]:
# Storing the dimensions
m, n = data.shape

# m - Number of images; n - label + pixels for each image
m, n


(42000, 785)

Splitting the data into dev and training. We're using dev to cross validate and we're setting aside only 1000 images to do so.

In [35]:
# Transposing the data using only 1000 images
data_dev = data[:1000].T

# Storing the labels in YDev
Y_dev = data_dev[0]

# Storing the pixels
X_dev = data_dev[1:]


In [36]:
# Storing the rest of the images
data_train = data[1000:m].T

# Extract labels
Y_train = data_train[0]

# Get the rest
X_train = data_train[1:]
X_train = X_train / 255


Printing details of all arrays implemented so far.

In [37]:
print(
    f'Printing dimensions of all existing arrays:\n(i) X - pixels\nX_dev: {X_dev.shape}\nX_train: {X_train.shape}\n\n(ii) Y - labels\nY_dev: {Y_dev.shape}\nY_train: {Y_train.shape}')


Printing dimensions of all existing arrays:
(i) X - pixels
X_dev: (784, 1000)
X_train: (784, 41000)

(ii) Y - labels
Y_dev: (1000,)
Y_train: (41000,)


# 1. Defining Initial Parameters

Defining a function to initialise the neural network by creating random weights. We use `rand()` to obtain a random value between 0 and 1 and then we subtract from those values to make sure the range in which our random values lie is `[-0.5, 0.5]`. 

In [38]:
def init_params():
    # # There's 10 connections for each of the 784 nodes
    # W1 = np.random.rand(10, n - 1) - 0.5

    # # There's 10 biases
    # b1 = np.random.rand(10, 1) - 0.5

    # # Similarly,
    # # There's 10 connections to 10 output nodes
    # W2 = np.random.rand(10, 10) - 0.5
    # b2 = np.random.rand(10, 1) - 0.5

    # return W1, b1, W2, b2

    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2


Function implementing the ReLU (rectified linear unit) activation function.

In [39]:
def ReLU(Z):
    # Taking the maximum element-wise using numpy
    return np.maximum(0, Z)


Function implementing the softmax activation function

In [40]:
def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    
    # Returning the probability
    return A


# 2. Forward Propagation

Defining a function to implement forward propagation through the neural net.

In [41]:
def forward_propagation(W1, b1, W2, b2, X):
    # # Deactivated first layer
    # Z1 = W1.dot(X) + b1
    
    # # Activating Z1
    # A1 = ReLU(Z1)
    
    # # Creating the next layer's deactivated input
    # Z2 = W2.dot(A1) + b2
    
    # # Since the next layer is the output layer, we apply softmax
    # A2 = softmax(Z2)
    
    # return Z1, A1, Z2, A2

    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

Function to implement one-hot encoding of Y. This is to represent the target classes as an array instead of a label.

In [42]:
def one_hot_encode(Y):
    # Encoding
    one_hot_encoded_df = pd.get_dummies(Y)

    # Taking the transpose so the columns represent images
    one_hot_encoded_array = np.array(one_hot_encoded_df).T

    return one_hot_encoded_array


Test to illustrate the working of `one_hot_encode(Y)`.

In [43]:
test = Y_train[:20]
test


array([8, 0, 5, 8, 7, 1, 6, 7, 5, 6, 4, 8, 3, 6, 2, 2, 2, 8, 7, 6],
      dtype=int64)

In [44]:
df = pd.get_dummies(test).T
df

ls = np.array(df)
ls


array([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0]],
      dtype=uint8)

Comparing it to our function.

In [45]:
one_hot_encode(test)


array([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0]],
      dtype=uint8)

Alternative one-hot encoding function.

In [46]:
def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    
    one_hot_Y[np.arange(Y.size), Y] = 1
    
    one_hot_Y = one_hot_Y.T
    
    return one_hot_Y


Function to implement the derivative of ReLU.

In [47]:
def derivative_ReLU(Z):
    # Returning 1 if the value is greater than 0. This is because the slope of the `linear` thing is 1.
    return Z > 0


# 3. Back Propagation

Function to back propagate through the neural network to calculate the differences in the weights and biases. 

In [48]:
def ReLU_deriv(Z):
    return Z > 0

def back_propagation(Z1, A1, Z2, A2, W2, X, Y):
    m = Y.size
    # one_hot_encoded_Y = one_hot_encode(Y)

    # dZ2 = A2 - one_hot_encoded_Y
    
    # one_hot_Y = one_hot(Y)
    # dZ2 = A2 - one_hot_Y
    
    # dW2 = (1 / m) * dZ2.dot(A1.T)
    # db2 = (1 / m) * np.sum(dZ2)

    # dZ1 = W2.T.dot(dZ2) * derivative_ReLU(Z1)
    # dW1 = (1 / m) * dZ1.dot(X.T)
    # db1 = (1 / m) * np.sum(dZ1)
    
    # return dW1, db1, dW2, db2

    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

# 4. Updating Parameters

Function to update the parameters using learning rate `alpha`.

In [49]:
def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    # W1 = W1 - alpha * dW1
    # b1 = b1 - alpha * db1
    
    # W2 = W2 - alpha * dW2
    # b2 = b2 - alpha * db2
    
    # return W1, b1, W2, b2

    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2

# 5. Defining Gradient Descent

In [50]:
def get_predictions(A):
    # Returns the indices of the max values
    return np.argmax(A, 0)

Testing the `get_predictions()` function.

In [51]:
test, get_predictions(test)

(array([8, 0, 5, 8, 7, 1, 6, 7, 5, 6, 4, 8, 3, 6, 2, 2, 2, 8, 7, 6],
       dtype=int64),
 0)

In [52]:
def get_accuracy(predictions, Y):
    print(predictions, Y)
    
    return np.sum(predictions == Y) / Y.size

Gradient descent is an optimization algorithm which is commonly-used to train machine learning models and neural networks.  Training data helps these models learn over time, and the cost function within gradient descent specifically acts as a barometer, gauging its accuracy with each iteration of parameter updates. Until the function is close to or equal to zero, the model will continue to adjust its parameters to yield the smallest possible error. Once machine learning models are optimized for accuracy, they can be powerful tools for artificial intelligence (AI) and computer science applications. 

In [53]:
def gradient_descent(X, Y, epochs, alpha):
    # Defining weights and biases
    # W1, b1, W2, b2 = init_params()

    # for i in range(epochs):
    #     # Step 1
    #     Z1, A1, Z2, A2 = forward_propagation(W1, b1, W2, b2, X)
        
    #     # Step 2
    #     dW1, db1, dW2, db2 = back_propagation(Z1, A1, Z2, A2, W2, X, Y)
        
    #     # Step 3
    #     W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        
    #     # Every 10th iteration
    #     if i % 10 == 0:
    #         print("Iteration: ", i)
            
    #         # A2 is the output from the forward propagation
    #         predictions = get_predictions(A2)
            
    #         print(get_accuracy(predictions, Y))
    
    # return W1, b1, W2, b2

    W1, b1, W2, b2 = init_params()
    for i in range(epochs):
        Z1, A1, Z2, A2 = forward_propagation(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = back_propagation(Z1, A1, Z2, A2, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

# 6. Running the neural network

In [54]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 500, 0.1)

Iteration:  0
[2 7 7 ... 0 0 0] [8 0 5 ... 6 0 3]
0.10078048780487806
Iteration:  10
[2 7 9 ... 4 0 0] [8 0 5 ... 6 0 3]
0.20131707317073172
Iteration:  20
[2 7 9 ... 4 0 1] [8 0 5 ... 6 0 3]
0.28573170731707315
Iteration:  30
[2 0 7 ... 4 0 1] [8 0 5 ... 6 0 3]
0.3539024390243902
Iteration:  40
[8 0 7 ... 6 0 1] [8 0 5 ... 6 0 3]
0.40929268292682924
Iteration:  50
[8 0 5 ... 6 0 1] [8 0 5 ... 6 0 3]
0.4617317073170732
Iteration:  60
[8 0 5 ... 6 0 1] [8 0 5 ... 6 0 3]
0.5089756097560976
Iteration:  70
[8 0 5 ... 6 0 8] [8 0 5 ... 6 0 3]
0.5497317073170732
Iteration:  80
[8 0 5 ... 6 0 8] [8 0 5 ... 6 0 3]
0.5806097560975609
Iteration:  90
[8 0 5 ... 6 0 8] [8 0 5 ... 6 0 3]
0.6091951219512195
Iteration:  100
[8 0 5 ... 6 0 8] [8 0 5 ... 6 0 3]
0.634
Iteration:  110
[8 0 5 ... 6 0 8] [8 0 5 ... 6 0 3]
0.6561463414634147
Iteration:  120
[8 0 5 ... 6 0 8] [8 0 5 ... 6 0 3]
0.6760975609756098
Iteration:  130
[8 0 5 ... 6 0 8] [8 0 5 ... 6 0 3]
0.6951463414634146
Iteration:  140
[8 0 5 ...